## Deliverable 2. Create a Customer Travel Destinations Map.

In [22]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [23]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Camacha,PT,33.0833,-16.3333,71.53,78,75,11.50,broken clouds
1,1,Faanui,PF,-16.4833,-151.7500,78.78,75,55,12.50,broken clouds
2,2,Sainte-Thecle,CA,46.8168,-72.4991,77.05,76,100,6.78,overcast clouds
3,3,Hobart,AU,-42.8794,147.3294,50.25,88,75,1.01,broken clouds
4,4,Mandalgovi,MN,45.7625,106.2708,62.80,67,79,4.50,broken clouds


In [25]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature for the trip?"))
max_temp = float(input("What is the maximum temperature for the trip?"))

What is the minimum temperature for the trip?70
What is the maximum temperature for the trip?80


In [26]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
city_data_temp_df = city_data_df.loc[(city_data_df["Max Temp"]>= min_temp) & (city_data_df["Max Temp"]<=max_temp) ]
city_data_temp_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Camacha,PT,33.0833,-16.3333,71.53,78,75,11.50,broken clouds
1,1,Faanui,PF,-16.4833,-151.7500,78.78,75,55,12.50,broken clouds
2,2,Sainte-Thecle,CA,46.8168,-72.4991,77.05,76,100,6.78,overcast clouds
9,9,Puerto Ayora,EC,-0.7393,-90.3518,76.96,84,43,8.01,scattered clouds
10,10,Salalah,OM,17.0151,54.0924,78.89,83,75,4.61,broken clouds


In [27]:
# 4a. Determine if there are any empty rows.
city_data_temp_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [28]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = city_data_temp_df.dropna()
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [29]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Camacha,PT,71.53,broken clouds,33.0833,-16.3333,
1,Faanui,PF,78.78,broken clouds,-16.4833,-151.7500,
2,Sainte-Thecle,CA,77.05,overcast clouds,46.8168,-72.4991,
9,Puerto Ayora,EC,76.96,scattered clouds,-0.7393,-90.3518,
10,Salalah,OM,78.89,broken clouds,17.0151,54.0924,
12,Bandarban,BD,76.35,overcast clouds,22.1965,92.2262,
18,Rikitea,PF,74.08,light rain,-23.1203,-134.9692,
21,Paita,PE,77.56,clear sky,-5.0892,-81.1144,
33,Thompson,CA,71.76,scattered clouds,55.7435,-97.8558,
34,Hobyo,SO,76.57,scattered clouds,5.3505,48.5268,


In [34]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    params["location"] = f"{lat},{lon}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    #url = https://maps.googleapis.com/maps/api/directions/json? + "&origin=" + city_origin 
       # + "&destination=" +city_destination
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
 

    # 6e. Make request and retrieve the JSON data from the search.
    
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found... skipping.")
    

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [36]:
# 7. Drop the rows where there is no Hotel Name.
hotel_results_df = hotel_df.dropna()
hotel_results_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Camacha,PT,71.53,broken clouds,33.0833,-16.3333,Hotel Porto Santo & Spa
1,Faanui,PF,78.78,broken clouds,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
2,Sainte-Thecle,CA,77.05,overcast clouds,46.8168,-72.4991,Camping Lac et Forêt
9,Puerto Ayora,EC,76.96,scattered clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
10,Salalah,OM,78.89,broken clouds,17.0151,54.0924,HAMDAN PLAZA HOTEL SALALAH


In [38]:
# 8a. Create the output File (CSV)

hotel_search_output_file = "WeatherPy_vaction.csv"


# 8b. Export the City_Data into a csv
hotel_results_df.to_csv(hotel_search_output_file, index_label="City_ID")

In [40]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_results_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_results_df[["Lat", "Lng"]]

In [41]:
# 11a. Add a marker layer for each city to the map. 

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))